In [1]:
%matplotlib notebook
from ipywidgets import interact, interactive, fixed, interact_manual, Box, widgets
import sys
sys.path.insert(1, "../bin/")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from radioml import *
from training import *
from functools import partial
from numpy import exp, pi, arange, sin, cos
import warnings
warnings.filterwarnings("ignore")


In [2]:
dataset_name = '2016.10a'
seed = 2016
set_seed(seed)
classes, data = load_numpy(os.path.join('../data/', dataset_name + '.h5')) 
train, validate, test = split_training(data, 0.5, 0.5)

df = pd.DataFrame()
df['ms'] = validate['ms']
df['snr'] = validate['snr']


In [3]:
## Write the function to plot PSD and IQ data from a j value

def snr_to_signal(j, data):
    sig = data["iq_data"][j]
    ms = data["ms"][j]
    snr = data["snr"][j]
    return sig, snr, ms

def psd_and_iq(index, noise_dB, normalized_freq):
    plt.close()
     
    sig, snr, ms = snr_to_signal(j = index)
    
    i = sig[0]
    q = sig[1]
    
    ## 
    x = 2
    y = 2
    ##
    
    ## Let's print the original first
    plt.subplot(x, y, 1)
    plt.psd(i + 1j*q)
    plt.title(f" PSD of the original {classes[ms]} signal at {snr}dB")
    plt.show()
    
    plt.subplot(x, y, 2)
    t = np.arange(0,128)
    plt.plot(t, i) 
    plt.plot(t, q)
    plt.title(f"{classes[ms]} raw IQ at {snr}dB")
    
    
    ## Now let's show the augmented signal
    aug_sig = noise_and_freq(sig, noise_dB, normalized_freq)
    aug_i = aug_sig[0]
    aug_q = aug_sig[1]
    plt.subplot(x, y, 3)
    plt.psd(aug_i + 1j*aug_q)
    plt.title(f" PSD of the Noisy {classes[ms]} signal at {snr}dB")
    plt.show()
    
    plt.subplot(x, y, 4)
    t = np.arange(0,128)
    plt.plot(t, aug_i) 
    plt.plot(t, aug_q)
    plt.title(f"{classes[ms]} augmented IQ at {snr}dB")
    
    plt.tight_layout()
    plt.show()

snr_to_signal = partial(snr_to_signal, data=data)


In [4]:
def noise_and_freq(sig, noise_dB, normalized_freq):
    """
    AWGN channel
    Add AWGN noise to input signal. The function adds AWGN noise vector to signal 's' to generate a resulting signal vector 'r' of specified SNR in dB. It also
    returns the noise vector 'n' that is added to the signal 's' and the power spectral density N0 of noise added
    Parameters:
        s : input/transmitted signal vector
        SNRdB : desired signal to noise ratio (expressed in dB) for the received signal
        L : oversampling factor (applicable for waveform simulation) default L = 1.
    Returns:
        r : received signal vector (r=s+n)
"""
# s is the original IQ data
    L=1
    Fs = 1000000
    gamma = 10**(noise_dB/10) #SNR to linear scale
    P=L*sum(sum(abs(sig)**2))/len(sig) # if s is a matrix [MxN]
    N0=P/gamma # Find the noise spectral density
    n = np.sqrt(N0/2)*(np.random.standard_normal(sig.shape))
    waveform = sig + n # received signal
    output_waveform = np.zeros((128, 128))
    
    
    output_waveform[0] = np.cos(waveform[0]*2*pi*(normalized_freq)*arange(len(waveform[0])))
    output_waveform[1] = np.sin(waveform[1]*2*pi*(normalized_freq)*arange(len(waveform[1])))
    
    return output_waveform


index = widgets.IntSlider(
    value=0,
    min=0,
    max=len(data["iq_data"])-1,
    step=1,
    description='IQ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)
noise = widgets.IntSlider(
    value=0,
    min=-100,
    max=100,
    step=10,
    description='Noise(in dB): ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)
CFO = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='CFO',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)
widgets.interact(
    psd_and_iq,
    index = index,
    noise_dB = noise,
    normalized_freq = CFO
);

## Build this interactive bar thingy
items = [index, noise, CFO]
box = Box(children=items)

interactive(children=(IntSlider(value=0, continuous_update=False, description='IQ', max=219999), IntSlider(val…